# Корневые эвристики

## Введение в корневые эвристики

**Определение:** Корневые эвристики (или sqrt-эвристики) — это класс алгоритмических методов, основанных на разбиении данных на блоки размера $O(\sqrt{n})$, где $n$ — размер входных данных.

Корневые эвристики позволяют достичь компромисса между временем выполнения запросов и временем обновления данных. Они особенно эффективны в задачах, где требуется поддерживать структуру данных с операциями обновления и запросами.

**Основная идея:** Разбить массив или другую структуру данных на блоки размера $\sqrt{n}$, что позволяет:
1. Выполнять запросы за $O(\sqrt{n})$
2. Выполнять обновления за $O(\sqrt{n})$

Такой подход часто оказывается оптимальным, когда невозможно достичь логарифмической сложности для обоих типов операций.


## Теоретическое обоснование

**Теорема 1:** Пусть имеется массив $A$ размера $n$. Если разбить его на блоки размера $k$, то количество блоков будет равно $\lceil \frac{n}{k} \rceil$. Оптимальный размер блока $k = \sqrt{n}$ минимизирует сумму $k + \frac{n}{k}$.

**Доказательство:**

Рассмотрим функцию $f(k) = k + \frac{n}{k}$, где $k$ — размер блока. Найдем минимум этой функции, взяв производную и приравняв её к нулю:

$$f'(k) = 1 - \frac{n}{k^2} = 0$$

Отсюда:
$$\frac{n}{k^2} = 1$$
$$k^2 = n$$
$$k = \sqrt{n}$$

Вторая производная $f''(k) = \frac{2n}{k^3} > 0$ для $k > 0$, что подтверждает, что это минимум.

Таким образом, при $k = \sqrt{n}$ функция $f(k) = k + \frac{n}{k} = \sqrt{n} + \frac{n}{\sqrt{n}} = 2\sqrt{n}$ достигает минимума.

$\blacksquare$

**Следствие:** При разбиении массива на блоки размера $\sqrt{n}$, и время запроса, и время обновления будут иметь сложность $O(\sqrt{n})$.


## Корневая декомпозиция массива

Рассмотрим классическую задачу: дан массив $A$ размера $n$, требуется обрабатывать запросы двух типов:
1. Обновить элемент $A[i]$ на значение $v$
2. Найти сумму элементов на отрезке $[L, R]$

**Наивное решение:**
- Обновление: $O(1)$
- Запрос суммы: $O(n)$

**Решение с использованием дерева отрезков:**
- Обновление: $O(\log n)$
- Запрос суммы: $O(\log n)$

**Решение с использованием корневой декомпозиции:**

Разобьем массив на блоки размера $\sqrt{n}$. Для каждого блока будем хранить сумму элементов в нем.

- Обновление: $O(1)$ для обновления элемента и пересчета суммы блока
- Запрос суммы: $O(\sqrt{n})$ в худшем случае (обработка до $2\sqrt{n}$ блоков и элементов)

**Теорема 2:** Алгоритм корневой декомпозиции для задачи о сумме на отрезке имеет сложность $O(1)$ для операции обновления и $O(\sqrt{n})$ для операции запроса суммы.

**Доказательство:**

1. **Обновление:** При обновлении элемента $A[i]$ на значение $v$, мы изменяем сам элемент и пересчитываем сумму соответствующего блока. Обе операции выполняются за $O(1)$.

2. **Запрос суммы:** При запросе суммы на отрезке $[L, R]$ мы:
   - Обрабатываем до $\sqrt{n}$ элементов в начальном неполном блоке
   - Обрабатываем до $\sqrt{n}$ полных блоков
   - Обрабатываем до $\sqrt{n}$ элементов в конечном неполном блоке

   Таким образом, общая сложность запроса составляет $O(\sqrt{n})$.

$\blacksquare$


In [ ]:
class SqrtDecomposition:
    def __init__(self, arr):
        self.arr = arr
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))
        self.num_blocks = (self.n + self.block_size - 1) // self.block_size
        self.block_sums = [0] * self.num_blocks

        # Предварительный расчет сумм блоков
        for i in range(self.n):
            self.block_sums[i // self.block_size] += arr[i]

    def update(self, idx, val):
        """Обновляет значение элемента и пересчитывает сумму блока."""
        block_idx = idx // self.block_size
        self.block_sums[block_idx] -= self.arr[idx]
        self.arr[idx] = val
        self.block_sums[block_idx] += val

    def query_sum(self, left, right):
        """Вычисляет сумму на отрезке [left, right]."""
        sum_result = 0

        # Обработка начального неполного блока
        start_block = left // self.block_size
        end_block = right // self.block_size

        if start_block == end_block:
            # Если запрос находится внутри одного блока
            for i in range(left, right + 1):
                sum_result += self.arr[i]
            return sum_result

        # Суммируем элементы начального неполного блока
        block_end = (start_block + 1) * self.block_size - 1
        for i in range(left, block_end + 1):
            sum_result += self.arr[i]

        # Суммируем полные блоки
        for i in range(start_block + 1, end_block):
            sum_result += self.block_sums[i]

        # Суммируем элементы конечного неполного блока
        block_start = end_block * self.block_size
        for i in range(block_start, right + 1):
            sum_result += self.arr[i]

        return sum_result

# Пример использования
arr = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
sqrt_decomp = SqrtDecomposition(arr)

print("Исходный массив:", arr)
print("Сумма на отрезке [2, 7]:", sqrt_decomp.query_sum(2, 7))

sqrt_decomp.update(4, 100)
print("Массив после обновления arr[4] = 100:", sqrt_decomp.arr)
print("Сумма на отрезке [2, 7] после обновления:", sqrt_decomp.query_sum(2, 7))

## Корневая эвристика для задачи RMQ (Range Minimum Query)

Задача RMQ (Range Minimum Query) заключается в нахождении минимального элемента на заданном отрезке массива.

**Определение:** Задача RMQ для массива $A$ размера $n$ состоит в обработке запросов вида "найти минимальный элемент на отрезке $[L, R]$".

**Теорема 3:** Алгоритм корневой декомпозиции для задачи RMQ имеет сложность $O(1)$ для операции обновления и $O(\sqrt{n})$ для операции запроса минимума.

**Доказательство:**

Аналогично доказательству для задачи о сумме на отрезке, при разбиении массива на блоки размера $\sqrt{n}$ и хранении минимума для каждого блока:

1. **Обновление:** При обновлении элемента $A[i]$ на значение $v$, мы изменяем сам элемент и пересчитываем минимум соответствующего блока за $O(1)$ или $O(\sqrt{n})$ в зависимости от реализации.

2. **Запрос минимума:** При запросе минимума на отрезке $[L, R]$ мы:
   - Находим минимум среди до $\sqrt{n}$ элементов в начальном неполном блоке
   - Находим минимум среди до $\sqrt{n}$ предварительно вычисленных минимумов полных блоков
   - Находим минимум среди до $\sqrt{n}$ элементов в конечном неполном блоке

   Таким образом, общая сложность запроса составляет $O(\sqrt{n})$.

$\blacksquare$


In [ ]:
class SqrtDecompositionRMQ:
    def __init__(self, arr):
        self.arr = arr
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))
        self.num_blocks = (self.n + self.block_size - 1) // self.block_size
        self.block_mins = [float('inf')] * self.num_blocks

        # Предварительный расчет минимумов блоков
        for i in range(self.n):
            block_idx = i // self.block_size
            self.block_mins[block_idx] = min(self.block_mins[block_idx], arr[i])

    def update(self, idx, val):
        """Обновляет значение элемента и пересчитывает минимум блока."""
        self.arr[idx] = val

        # Пересчитываем минимум блока
        block_idx = idx // self.block_size
        block_start = block_idx * self.block_size
        block_end = min(block_start + self.block_size, self.n)

        self.block_mins[block_idx] = float('inf')
        for i in range(block_start, block_end):
            self.block_mins[block_idx] = min(self.block_mins[block_idx], self.arr[i])

    def query_min(self, left, right):
        """Находит минимум на отрезке [left, right]."""
        min_val = float('inf')

        # Обработка начального неполного блока
        start_block = left // self.block_size
        end_block = right // self.block_size

        if start_block == end_block:
            # Если запрос находится внутри одного блока
            for i in range(left, right + 1):
                min_val = min(min_val, self.arr[i])
            return min_val

        # Находим минимум в начальном неполном блоке
        block_end = (start_block + 1) * self.block_size - 1
        for i in range(left, block_end + 1):
            min_val = min(min_val, self.arr[i])

        # Находим минимум среди полных блоков
        for i in range(start_block + 1, end_block):
            min_val = min(min_val, self.block_mins[i])

        # Находим минимум в конечном неполном блоке
        block_start = end_block * self.block_size
        for i in range(block_start, right + 1):
            min_val = min(min_val, self.arr[i])

        return min_val

# Пример использования
arr = [9, 3, 7, 1, 8, 12, 10, 20, 15, 13]
rmq = SqrtDecompositionRMQ(arr)

print("Исходный массив:", arr)
print("Минимум на отрезке [2, 7]:", rmq.query_min(2, 7))

rmq.update(4, 2)
print("Массив после обновления arr[4] = 2:", rmq.arr)
print("Минимум на отрезке [2, 7] после обновления:", rmq.query_min(2, 7))

## Алгоритм Мо (Mo's Algorithm)

Алгоритм Мо — это метод обработки оффлайн-запросов к массиву с использованием корневой эвристики. Он особенно эффективен, когда запросы известны заранее и не требуют обновлений массива.

**Определение:** Алгоритм Мо для обработки запросов к массиву $A$ размера $n$ заключается в сортировке запросов по специальному правилу и последовательной их обработке с поддержанием текущего состояния.

**Теорема 4:** Алгоритм Мо для обработки $m$ запросов к массиву размера $n$ имеет временную сложность $O((n+m) \cdot \sqrt{n})$.

**Доказательство:**

1. Разобьем массив на блоки размера $\sqrt{n}$.
2. Отсортируем запросы по номеру блока левой границы, а при равенстве — по правой границе.
3. При обработке запросов будем поддерживать текущее состояние для отрезка $[L, R]$ и модифицировать его, добавляя или удаляя элементы для перехода к следующему запросу.

Оценим количество операций:
- Сортировка $m$ запросов: $O(m \log m)$
- Переходы между запросами: в худшем случае мы делаем $O(n)$ операций для каждого запроса, но благодаря сортировке общее количество операций будет $O((n+m) \cdot \sqrt{n})$

$\blacksquare$


In [ ]:
class MoAlgorithm:
    def __init__(self, arr):
        self.arr = arr
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))

    def process_queries(self, queries):
        """
        Обрабатывает запросы с использованием алгоритма Мо.
        Каждый запрос представлен парой (left, right).
        Возвращает список результатов (сумм на отрезках).
        """
        # Добавляем индекс к каждому запросу для отслеживания порядка
        indexed_queries = [(i, left, right) for i, (left, right) in enumerate(queries)]

        # Сортируем запросы по блоку левой границы, затем по правой границе
        indexed_queries.sort(key=lambda q: (q[1] // self.block_size, q[2]))

        results = [0] * len(queries)
        current_sum = 0
        current_left = 0
        current_right = -1

        for idx, left, right in indexed_queries:
            # Расширяем текущий отрезок вправо
            while current_right < right:
                current_right += 1
                current_sum += self.arr[current_right]

            # Сужаем текущий отрезок справа
            while current_right > right:
                current_sum -= self.arr[current_right]
                current_right -= 1

            # Расширяем текущий отрезок влево
            while current_left > left:
                current_left -= 1
                current_sum += self.arr[current_left]

            # Сужаем текущий отрезок слева
            while current_left < left:
                current_sum -= self.arr[current_left]
                current_left += 1

            # Сохраняем результат для текущего запроса
            results[idx] = current_sum

        return results

# Пример использования
arr = [1, 5, 3, 7, 9, 2, 6, 4, 8]
mo = MoAlgorithm(arr)

queries = [(0, 3), (1, 5), (3, 7), (2, 8)]
results = mo.process_queries(queries)

print("Исходный массив:", arr)
print("Запросы (left, right):", queries)
print("Результаты (суммы на отрезках):", results)

## Корневая эвристика для задачи о ближайшем меньшем элементе

Рассмотрим задачу о нахождении ближайшего меньшего элемента для каждого элемента массива.

**Определение:** Для массива $A$ размера $n$ требуется для каждого элемента $A[i]$ найти индекс $j < i$ такой, что $A[j] < A[i]$ и $j$ максимален (ближайший меньший слева).

**Теорема 5:** Алгоритм с использованием корневой эвристики для задачи о ближайшем меньшем элементе имеет временную сложность $O(n \cdot \sqrt{n})$.

**Доказательство:**

1. Разобьем массив на блоки размера $\sqrt{n}$.
2. Для каждого блока предварительно вычислим для каждой возможной высоты $h$ ближайший элемент меньше $h$ в этом блоке.
3. Для каждого элемента $A[i]$ найдем ближайший меньший элемент, рассматривая:
   - Элементы в текущем блоке слева от $A[i]$
   - Предварительно вычисленные результаты для предыдущих блоков

Предварительные вычисления для всех блоков требуют $O(n \cdot \sqrt{n})$ операций.
Обработка каждого элемента требует $O(\sqrt{n})$ операций.
Общая сложность: $O(n \cdot \sqrt{n})$.

$\blacksquare$


In [ ]:
def nearest_smaller_element(arr):
    """
    Находит для каждого элемента массива ближайший меньший элемент слева.
    Возвращает массив индексов (или -1, если такого элемента нет).
    """
    n = len(arr)
    result = [-1] * n

    # Наивное решение за O(n²)
    for i in range(1, n):
        for j in range(i-1, -1, -1):
            if arr[j] < arr[i]:
                result[i] = j
                break

    return result

def nearest_smaller_element_sqrt(arr):
    """
    Находит для каждого элемента массива ближайший меньший элемент слева
    с использованием корневой эвристики.
    """
    n = len(arr)
    result = [-1] * n
    block_size = int(math.sqrt(n))
    num_blocks = (n + block_size - 1) // block_size

    # Предварительные вычисления для каждого блока
    block_data = [[] for _ in range(num_blocks)]

    for block in range(num_blocks):
        start = block * block_size
        end = min(start + block_size, n)

        # Для каждого элемента в блоке находим ближайший меньший в этом же блоке
        for i in range(start, end):
            nearest = -1
            for j in range(i-1, start-1, -1):
                if arr[j] < arr[i]:
                    nearest = j
                    break

            if i > start:  # Не первый элемент блока
                result[i] = nearest

    # Обработка межблочных зависимостей
    for block in range(1, num_blocks):
        start = block * block_size
        end = min(start + block_size, n)

        for i in range(start, end):
            # Если не нашли ближайший меньший в текущем блоке
            if result[i] == -1 or result[i] < start:
                # Ищем в предыдущих блоках
                for prev_block in range(block-1, -1, -1):
                    prev_start = prev_block * block_size
                    prev_end = prev_block * block_size + block_size

                    # Ищем ближайший меньший в предыдущем блоке
                    for j in range(prev_end-1, prev_start-1, -1):
                        if j >= n:
                            continue
                        if arr[j] < arr[i]:
                            result[i] = j
                            break

                    if result[i] != -1 and result[i] >= prev_start:
                        break

    return result

# Пример использования
arr = [4, 2, 1, 5, 6, 3, 2, 4, 2]
result_naive = nearest_smaller_element(arr)
result_sqrt = nearest_smaller_element_sqrt(arr)

print("Исходный массив:", arr)
print("Ближайшие меньшие элементы (индексы) - наивное решение:", result_naive)
print("Ближайшие меньшие элементы (индексы) - sqrt-эвристика:", result_sqrt)

## Применение корневых эвристик в задачах с обновлениями на отрезке

Рассмотрим задачу с обновлениями на отрезке: дан массив $A$ размера $n$, требуется обрабатывать запросы двух типов:
1. Добавить значение $v$ ко всем элементам на отрезке $[L, R]$
2. Найти значение элемента $A[i]$

**Теорема 6:** Алгоритм с использованием корневой эвристики для задачи с обновлениями на отрезке имеет временную сложность $O(\sqrt{n})$ для обоих типов запросов.

**Доказательство:**

1. Разобьем массив на блоки размера $\sqrt{n}$.
2. Для каждого блока будем хранить значение добавки (lazy propagation).
3. При запросе обновления на отрезке $[L, R]$:
   - Для полностью покрытых блоков обновляем только значение добавки
   - Для частично покрытых блоков обновляем каждый элемент отдельно
4. При запросе значения элемента $A[i]$:
   - Возвращаем исходное значение элемента плюс значение добавки для его блока

Сложность запроса обновления: $O(\sqrt{n})$ (до $2\sqrt{n}$ элементов в частично покрытых блоках и до $\sqrt{n}$ полных блоков).
Сложность запроса значения: $O(1)$.

$\blacksquare$


In [ ]:
class SqrtDecompositionWithRangeUpdates:
    def __init__(self, arr):
        self.arr = arr.copy()  # Исходный массив
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))
        self.num_blocks = (self.n + self.block_size - 1) // self.block_size
        self.lazy = [0] * self.num_blocks  # Значения добавок для блоков

    def update_range(self, left, right, val):
        """Добавляет значение val ко всем элементам на отрезке [left, right]."""
        start_block = left // self.block_size
        end_block = right // self.block_size

        # Обработка частично покрытого начального блока
        if start_block == end_block:
            # Если запрос находится внутри одного блока
            for i in range(left, right + 1):
                self.arr[i] += val
            return

        # Обработка начального неполного блока
        block_end = (start_block + 1) * self.block_size - 1
        for i in range(left, block_end + 1):
            self.arr[i] += val

        # Обработка полных блоков
        for i in range(start_block + 1, end_block):
            self.lazy[i] += val

        # Обработка конечного неполного блока
        block_start = end_block * self.block_size
        for i in range(block_start, right + 1):
            self.arr[i] += val

    def get_value(self, idx):
        """Возвращает текущее значение элемента с индексом idx."""
        block_idx = idx // self.block_size
        return self.arr[idx] + self.lazy[block_idx]

# Пример использования
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sqrt_decomp = SqrtDecompositionWithRangeUpdates(arr)

print("Исходный массив:", arr)
print("Значение элемента с индексом 3:", sqrt_decomp.get_value(3))

sqrt_decomp.update_range(2, 7, 10)
print("Значение элемента с индексом 3 после обновления отрезка [2, 7] на +10:", sqrt_decomp.get_value(3))
print("Значение элемента с индексом 0 после обновления:", sqrt_decomp.get_value(0))
print("Значение элемента с индексом 9 после обновления:", sqrt_decomp.get_value(9))

## Заключение

Корневые эвристики представляют собой мощный инструмент для разработки эффективных алгоритмов, особенно в задачах с запросами на отрезках и обновлениями. Основные преимущества корневых эвристик:

1. **Простота реализации** по сравнению с более сложными структурами данных, такими как дерево отрезков или дерево Фенвика.

2. **Компромисс между временем запроса и обновления**, что делает их особенно полезными в задачах, где невозможно достичь логарифмической сложности для обоих типов операций.

3. **Эффективность в практических задачах**, особенно когда требуется баланс между временем запроса и временем обновления.

4. **Хорошая локальность данных**, что может приводить к лучшей производительности на реальных компьютерах из-за эффективного использования кэша.

Корневые эвристики нашли применение во многих областях алгоритмики, включая:
- Обработку запросов на отрезках
- Геометрические алгоритмы
- Строковые алгоритмы
- Алгоритмы на графах

Несмотря на то, что в теоретическом плане существуют более эффективные структуры данных (например, дерево отрезков с асимптотикой $O(\log n)$ для запросов и обновлений), корневые эвристики часто оказываются предпочтительными из-за своей простоты и эффективности на практике.


## Практические применения

### Задача о количестве различных элементов на отрезке

Рассмотрим задачу: дан массив $A$ размера $n$ и $m$ запросов вида "найти количество различных элементов на отрезке $[L, R]$".

Эта задача эффективно решается с помощью алгоритма Мо:


In [ ]:
from collections import defaultdict

class MoAlgorithmDistinct:
    def __init__(self, arr):
        self.arr = arr
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))

    def count_distinct(self, queries):
        """
        Обрабатывает запросы на подсчет различных элементов на отрезках.
        Каждый запрос представлен парой (left, right).
        """
        # Добавляем индекс к каждому запросу
        indexed_queries = [(i, left, right) for i, (left, right) in enumerate(queries)]

        # Сортируем запросы по блоку левой границы, затем по правой границе
        indexed_queries.sort(key=lambda q: (q[1] // self.block_size, q[2]))

        results = [0] * len(queries)
        current_distinct = 0
        frequency = defaultdict(int)
        current_left = 0
        current_right = -1

        for idx, left, right in indexed_queries:
            # Расширяем текущий отрезок вправо
            while current_right < right:
                current_right += 1
                frequency[self.arr[current_right]] += 1
                if frequency[self.arr[current_right]] == 1:
                    current_distinct += 1

            # Сужаем текущий отрезок справа
            while current_right > right:
                frequency[self.arr[current_right]] -= 1
                if frequency[self.arr[current_right]] == 0:
                    current_distinct -= 1
                current_right -= 1

            # Расширяем текущий отрезок влево
            while current_left > left:
                current_left -= 1
                frequency[self.arr[current_left]] += 1
                if frequency[self.arr[current_left]] == 1:
                    current_distinct += 1

            # Сужаем текущий отрезок слева
            while current_left < left:
                frequency[self.arr[current_left]] -= 1
                if frequency[self.arr[current_left]] == 0:
                    current_distinct -= 1
                current_left += 1

            # Сохраняем результат для текущего запроса
            results[idx] = current_distinct

        return results

# Пример использования
arr = [1, 2, 1, 3, 2, 1, 4, 5, 3, 2]
mo_distinct = MoAlgorithmDistinct(arr)

queries = [(0, 3), (2, 7), (1, 9), (5, 9)]
results = mo_distinct.count_distinct(queries)

print("Исходный массив:", arr)
print("Запросы (left, right):", queries)
print("Количество различных элементов на отрезках:", results)


### Задача о медиане на отрезке

Рассмотрим задачу: дан массив $A$ размера $n$ и $m$ запросов вида "найти медиану на отрезке $[L, R]$".

Эта задача также эффективно решается с помощью алгоритма Мо:


In [ ]:
import bisect

class MoAlgorithmMedian:
    def __init__(self, arr):
        self.arr = arr
        self.n = len(arr)
        self.block_size = int(math.sqrt(self.n))

    def find_medians(self, queries):
        """
        Обрабатывает запросы на нахождение медианы на отрезках.
        Каждый запрос представлен парой (left, right).
        """
        # Добавляем индекс к каждому запросу
        indexed_queries = [(i, left, right) for i, (left, right) in enumerate(queries)]

        # Сортируем запросы по блоку левой границы, затем по правой границе
        indexed_queries.sort(key=lambda q: (q[1] // self.block_size, q[2]))

        results = [0] * len(queries)
        current_elements = []
        current_left = 0
        current_right = -1

        for idx, left, right in indexed_queries:
            # Расширяем текущий отрезок вправо
            while current_right < right:
                current_right += 1
                bisect.insort(current_elements, self.arr[current_right])

            # Сужаем текущий отрезок справа
            while current_right > right:
                current_elements.remove(self.arr[current_right])
                current_right -= 1

            # Расширяем текущий отрезок влево
            while current_left > left:
                current_left -= 1
                bisect.insort(current_elements, self.arr[current_left])

            # Сужаем текущий отрезок слева
            while current_left < left:
                current_elements.remove(self.arr[current_left])
                current_left += 1

            # Вычисляем медиану
            n = len(current_elements)
            if n % 2 == 1:
                median = current_elements[n // 2]
            else:
                median = (current_elements[n // 2 - 1] + current_elements[n // 2]) / 2

            # Сохраняем результат для текущего запроса
            results[idx] = median

        return results

# Пример использования
arr = [1, 3, 5, 7, 9, 2, 4, 6, 8, 10]
mo_median = MoAlgorithmMedian(arr)

queries = [(0, 4), (2, 7), (5, 9), (0, 9)]
results = mo_median.find_medians(queries)

print("Исходный массив:", arr)
print("Запросы (left, right):", queries)
print("Медианы на отрезках:", results)
